In [2]:
import os
os.chdir('..')

In [3]:
%pwd

'c:\\Users\\anasm\\OneDrive\\Desktop\\Image Processing\\Kidney disease detection'

In [4]:
#add to entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PreparePredictionConfig:
    root_dir: Path
    trained_model_path: Path
    prediction_data: Path

In [5]:
from src.constants import *
import tensorflow as tf
from src.utils.common import read_yaml

In [6]:
PREDICTION_DATA_PATH

WindowsPath('C:/Users/anasm/OneDrive/Desktop/Image Processing/Kidney disease detection/Prediction')

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH):
        self.config=read_yaml(config_filepath)

    def get_prediction_configs(self) -> PreparePredictionConfig:
        prediction_config=PreparePredictionConfig(
            root_dir=self.config.prepare_base_model.root_dir,
            trained_model_path=self.config.training.trained_model_path,
            prediction_data= PREDICTION_DATA_PATH
        )

        return prediction_config

In [8]:
cmanager=ConfigurationManager()
prediction_config=cmanager.get_prediction_configs()

[2024-08-18 00:30:19,635: INFO: common: yaml file: C:\Users\anasm\OneDrive\Desktop\Image Processing\Kidney disease detection\config\config.yaml loaded successfully]


In [9]:
import cv2
import numpy as np
class Predictor:
    def __init__(self, config: PreparePredictionConfig):
        self.config=config
    
    def load_model(self):
        self.model=tf.keras.models.load_model(self.config.trained_model_path)

    def load_data(self):
        self.data=[]
        for filename in os.listdir(self.config.prediction_data):
            filepath=os.path.join(self.config.prediction_data, filename)
            img=cv2.imread(filepath, 1)
            img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img=cv2.resize(img, (224, 224))
            self.data.append(img)
        self.data=np.array(self.data, dtype=np.uint8)

    def estimate(self):
        self.y=self.model.predict(self.data)
        return self.y
    

In [10]:
p=Predictor(prediction_config)
p.load_model()

[2024-08-18 00:30:19,896: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


In [11]:
p.load_data()

In [12]:
y=p.estimate()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step


In [13]:
print(y.argmax())

0
